In [1]:
import sys

sys.path.append("..")

from src.utils.auth_trees import get_user_clients, MySQL

In [2]:
mySQL = MySQL()

In [3]:
params = {
    "user_id": 18089,
    "permission_level": 0,
}

In [4]:
query = """SELECT DISTINCT client_id
FROM `api_collaborators`
WHERE `user_id` = %(user_id)s
ORDER BY client_id DESC;"""
params = {
    "user_id": params["user_id"],
}
collaborate_clients = mySQL.query_to_list(query, params)

In [5]:
placeholders = ', '.join(['%s'] * len(collaborate_clients))
query = f"""SELECT id, parent_id
FROM api_clients
WHERE parent_id IN ({placeholders})"""
clients = mySQL.execute_query_dict(
    query=query,
    params=collaborate_clients,
)

In [6]:
class Client:
    def __init__(self, id, parent_id, billing_client_id=None, clients=None):
        self.id = id
        self.parent_id = parent_id
        self.billing_client_id = billing_client_id
        self.clients = clients or []

    def set_relation(self, value):
        self.clients = value

    def __getattr__(self, name):
        # Fallback to _relations if accessed like client.clients
        if name in self._relations:
            return self._relations[name]
        raise AttributeError(f"{name} not found")


In [7]:
clients = [
    Client(
        client["id"],
        client["parent_id"],
        client.get("billing_client_id")
    )
    for client in clients
]

In [8]:
parents = mySQL.execute_query_dict(
    """SELECT id, parent_id
    FROM api_clients
    ORDER BY name"""
)

In [9]:
parents_lookup = {}
for client in parents:
    parent_id = client["parent_id"]
    if parent_id not in parents_lookup:
        parents_lookup[parent_id] = []
    parents_lookup[parent_id].append(
        Client(
            client["id"],
            client["parent_id"],
            client.get("billing_client_id")
        )
    )

In [10]:
def get_all_clients(clients, clients_list=None, cache=None, exclude_billing_to_other_clients=False):
    if clients_list is None:
        clients_list = []

    for client in clients:
        if exclude_billing_to_other_clients and client.billing_client_id is not None:
            continue

        clients_list.append(client)

        if cache is not None:
            if client.id in cache:
                client.set_relation(cache[client.id])
            else:
                client.set_relation([])

        # Recursive call
        get_all_clients(client.clients, clients_list, cache, exclude_billing_to_other_clients)

    return clients_list


In [11]:
clients_list = get_all_clients(
    clients,
    cache=parents_lookup,
)

In [16]:
for client in clients_list:
    for child in client.clients:
        if len(child.clients) > 0:
            print(f"Client {client.id} has child {child.id} with clients: {[c.id for c in child.clients]}")

Client 2 has child 59 with clients: [61, 60, 62]
Client 2 has child 6 with clients: [9, 8, 7, 10]
Client 12 has child 145 with clients: [877]
Client 13 has child 170 with clients: [26, 19, 111, 73, 456, 57, 27]
Client 13 has child 12108 with clients: [12109, 12110, 545]
Client 13 has child 9272 with clients: [19963, 13388, 16426, 14448, 14417, 14652, 17607]
Client 13 has child 168 with clients: [381, 418, 426, 7687, 1189, 22006, 1662, 10547, 10959, 11199, 10548, 9557, 8624, 2232, 2564, 1838, 2146, 12489, 3102, 7165, 7166, 18218, 1214, 262, 248, 2173, 828, 1641, 13994, 1889, 19806, 21099, 6220, 2565, 2950, 841, 338, 459, 12982, 18821, 2198, 3231, 2197, 1774, 868, 2381, 20520, 18970, 4746, 4027, 4476, 458, 12852, 9743, 20278, 7070, 6967, 3969, 13293, 1970, 928, 2887, 19334, 1945, 1850, 876, 602, 330, 8119, 11784, 4274, 12981, 474, 110, 17832, 9618, 7283, 7284, 627, 7181, 2521, 20335, 2765, 5823, 3206, 6034, 2683]
Client 13 has child 167 with clients: [2096, 9254, 19355, 10906, 11016, 840

In [19]:
clients_list[0].id

2

In [22]:
clients_list[0].clients[1].id

59